In [64]:
import re
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')

df = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv")

# Algemene Nederlandse stopwoorden
DUTCH_STOPWORDS = set(stopwords.words("dutch")) | {
    "bij", "voor", "met", "door", "zonder", "over",
    "doelgroep", "werk", "werken", "proces", "praktijk",
    "ontwikkeling", "ontwikkelen", "gaan",
    "leren", "school", "module", "thema",
    "student", "opleiding", "kun" , "vanuit",
    "eigen", "zelf", "samen", "samenwerken",
    "jaar", "week", "periode", "naasten", "daarnaast",
    "minor", "studenten", "programma", "keuzemodule",
    "casus", "casussen", "cases", "vraagstukken",
    "stage", "stageschool", "kennismakingsstage",
    "kennis", "vaardigheid", "vaardigheden", "ervaring", "ervaringen",
    "lessen", "onderwerpen", "theorie", "praktijk", "praktische", "inhoudelijke",
    "mee", "doe", "vinden", "vind", "kies", "openstaan",
    "belangrijk", "positief", "mogelijkheden","mogelijkheid",
    "impact", "betekenis", "betekent", "betekenen",
    "you", "your", "are", "will", "what", "then", "like", "choose",
    "interested", "experiencing", "hbo", "and", "the", "persoonlijke",
    "learning", "denken", "maken", "business", "verdieping", "emgeving",
    "bouwen", "thinking", "branding", "maken", "urban", "veiligheid",
    "nieuwe", "test", "gebouwde", "concept", "project", "omgeving", "actuele", "acute",
    "yellow", "belt", "serious", "hrm", "mensen", "snel", "binnen", "materialen", 
    "active", "druk", "context", "leven", "complexe" , "brede", "for", "jouw", "manieren"
}

def clean_text_for_matching(text: str) -> str:
    """Maak tekst schoon voor matching / uitleg."""
    text = str(text).lower()
    # haal alleen 'normale' woorden eruit
    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)
    # filter stopwoorden en superkorte zooi
    tokens = [
        t for t in tokens
        if t not in DUTCH_STOPWORDS and len(t) > 2
    ]
    return " ".join(tokens)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeroe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 📘 Feedback van de docent

**Hieronder staat gerichte feedback op jullie notebook, gebaseerd op de rubric en opdracht.**

**Wat gaat goed:**
- Jullie hebben een functionele content‑based recommender.
- De code is netjes opgebouwd en duidelijk leesbaar.
- Basis NLP‑cleaning en similarity werkt goed.

**Wat ontbreekt / moet beter voor de rubric:**
- **Geen analyse van prestaties** (bijvoorbeeld uitleg waarom keuzes gemaakt zijn).
- **Geen optimalisatie/tuning** (bijv. parameters TF-IDF, embeddings testen).

**Aanbevolen verbeteringen:**
3. Voeg uitleg toe bij elke stap: waarom deze keuze, hoe werkt het model?
4. Voeg experimenten toe met TF‑IDF parameters, of evt. embeddings.
5. Toon eindresultaten in een tabel + korte motivatie waarom deze modules passen.

In [65]:
# Als je al combined_text hebt, gebruik die.
if "combined_text" in df.columns:
    base_text = df["combined_text"]
else:
    # Fallback: zelf combineren (pas aan naar wat jij wilt gebruiken)
    df["combined_text"] = (
        df["name"].fillna("") + " " +
        df["shortdescription"].fillna("") + " " +
        df["description"].fillna("") + " " +
        df["content"].fillna("") + " " +
        df["module_tags"].astype(str).fillna("") +" " +
        df["location"].astype(str).fillna("")
    )
    base_text = df["combined_text"]

# Schoongemaakte tekst voor recommender + uitleg
df["clean_text"] = base_text.apply(clean_text_for_matching)


In [66]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean_text"])

FEATURE_NAMES = vectorizer.get_feature_names_out()

In [67]:
def extract_match_terms(student_vec, module_vec, feature_names, max_terms: int = 8):
    student_idx = set(student_vec.nonzero()[1])
    module_idx = set(module_vec.nonzero()[1])
    shared_idx = sorted(student_idx & module_idx)

    terms = [feature_names[i] for i in shared_idx]
    return terms[:max_terms]


In [68]:
import random

def _format_term_list(terms: list[str]) -> str:
    """Maak een nette NL opsomming: 'A', 'A en B', 'A, B en C'."""
    if not terms:
        return ""
    if len(terms) == 1:
        return f"'{terms[0]}'"
    if len(terms) == 2:
        return f"'{terms[0]}' en '{terms[1]}'"
    # 3 of meer
    quoted = [f"'{t}'" for t in terms]
    hoofd = ", ".join(quoted[:-1])
    laatste = quoted[-1]
    return f"{hoofd} en {laatste}"

def build_reason(match_terms: list[str], module_name: str | None = None, score: float | None = None) -> str:
    """
    Genereer een kort NL tekstje waarom de module past.
    - match_terms: termen waarop je matcht (interesses / keywords)
    - module_name: optioneel, naam van de module voor iets persoonlijkere tekst
    - score: optioneel, similarity-score (0–1) om bv. 'sterk' / 'goed' aan te sturen
    """
    terms_str = _format_term_list(match_terms)

    # score -> woordje voor sterkte (heel grof)
    if score is None:
        kwalificatie = "goed"
    elif score >= 0.8:
        kwalificatie = "erg goed"
    elif score >= 0.6:
        kwalificatie = "goed"
    else:
        kwalificatie = "redelijk"

    if not match_terms:
        # Geen specifieke termen: meer algemene tekst
        templates = [
            "Deze module sluit {kwalificatie} aan bij je interesses op basis van tekstuele overeenkomsten.",
            "Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module {kwalificatie} bij je te passen.",
            "Deze module lijkt inhoudelijk {kwalificatie} aan te sluiten bij wat je interessant vindt."
        ]
    else:
        # Met match_terms
        if module_name:
            templates = [
                "Deze module sluit {kwalificatie} aan bij je interesses in {terms}, zoals die terugkomen in '{module}'.",
                "Omdat {terms} centraal staan in '{module}', past deze module {kwalificatie} bij jouw interesses.",
                "Je interesse in {terms} komt duidelijk terug in '{module}', waardoor deze module {kwalificatie} bij je aansluit."
            ]
        else:
            templates = [
                "Deze module sluit {kwalificatie} aan bij je interesses in {terms}.",
                "Omdat {terms} in deze module aan bod komen, lijkt deze {kwalificatie} bij je te passen.",
                "Je interesse in {terms} komt terug in de inhoud van deze module, waardoor deze goed bij je past."
            ]

    template = random.choice(templates)
    return template.format(
        kwalificatie=kwalificatie,
        terms=terms_str,
        module=module_name if module_name else ""
    )


In [69]:
def recommend_modules( student_profile: str, top_n: int = 5, studycredit: int | None = None, level: str | None = None, location: str | None = None) -> pd.DataFrame:
    """
    Geeft top N modules voor een studentprofiel,
    inclusief similarity (genormaliseerd 0–1), match_terms en een NL-uitleg.
    """
    # Start met df + bijbehorende indices
    filtered_df = df.copy()

    # Filteren op studiepunten, niveau, locatie
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]
    if level is not None:
        filtered_df = filtered_df[filtered_df["level"].isin([level])]
    if location is not None:
        filtered_df = filtered_df[
            filtered_df["location"].str.contains(location, case=False, na=False)
        ]

    # Als er niks overblijft
    if filtered_df.empty:
        return filtered_df.assign(
            similarity=[],
            match_terms=[],
            reason=[]
        )

    # Studentprofiel vectoriseren
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    # Pak de juiste rijen uit de globale X aan de hand van de indexen van filtered_df
    row_indices = filtered_df.index.to_numpy()
    X_filtered = X[row_indices]

    # Cosine similarity
    sims = cosine_similarity(student_vec, X_filtered).flatten()

    # Ruwe similarity bewaren (optioneel, voor debug)
    filtered_df = filtered_df.copy()
    filtered_df["similarity_raw"] = sims

    # Normaliseren naar 0–1 binnen deze selectie
    max_sim = sims.max()
    if max_sim > 0:
        norm_sims = sims / max_sim
    else:
        norm_sims = sims  # alles 0

    filtered_df["similarity"] = norm_sims

    # Top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    match_terms_list = []
    reasons = []

    for idx in top.index:
        # positie van deze rij binnen X_filtered
        pos = np.where(row_indices == idx)[0][0]
        module_vec = X_filtered[pos]

        terms = extract_match_terms(student_vec, module_vec, FEATURE_NAMES)
        match_terms_list.append(terms)

        reasons.append(
            build_reason(
                terms,
                module_name=top.at[idx, "name"],
                score=top.at[idx, "similarity"]  # genormaliseerde score
            )
        )

    top = top[["id", "name", "shortdescription", "similarity", "similarity_raw", "location", "studycredit", "level", "module_tags",]].copy()
    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top


In [70]:
sam_profile = """
Ik ben derdejaarsstudent en ik heb veel interesse in zorginnovatie,
data en AI. Ik wil graag modules volgen over zorgtechnologie, 
innovatie in de zorg en creatieve manieren om met technologie impact te maken.
"""

# sam_studycredit = 30
# sam_level = "NLQF6"
# sam_location = "Breda"


sam_studycredit = None
sam_level = None
sam_location = None

pd.set_option("display.max_colwidth", None)

# aanbevelingen = recommend_modules(sam_profile, top_n=5, studycredit=sam_studycredit, level=sam_level, location=sam_location)
aanbevelingen = recommend_modules(sam_profile, top_n=5, )
display(aanbevelingen)


,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
17,176,Technologie die ècht werkt: innovatie in zorg en welzijn,"Technologie, smart health, robotica, zorg, welzijn",1.000000,0.220202,Breda,15,NLQF5,"['technologie', 'smart', 'health', 'robotica', 'zorg', 'welzijn']","[innovatie, technologie, zorg]","Deze module sluit erg goed aan bij je interesses in 'innovatie', 'technologie' en 'zorg', zoals die terugkomen in 'Technologie die ècht werkt: innovatie in zorg en welzijn'."
13,172,Zorg dichtbij,"Zorg op afstand, technologie, implementeren, zorg, veranderkunde",0.986076,0.217135,Breda,15,NLQF5,"['zorg', 'afstand', 'technologie', 'implementeren', 'veranderkunde']","[technologie, zorg]","Omdat 'technologie' en 'zorg' centraal staan in 'Zorg dichtbij', past deze module erg goed bij jouw interesses."
140,331,Innovatiemanagement,"Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken. Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken.",0.858365,0.189013,Tilburg,30,NLQF6,"['innovatie draait', 'innovatie plaatsvinden', 'innovatie verbeteringen', 'innovatie', 'processen innovatie', 'innovatie vernieuwing', 'voeren innovatie', 'vernieuwing invoeren']","[innovatie, technologie]","Deze module sluit erg goed aan bij je interesses in 'innovatie' en 'technologie', zoals die terugkomen in 'Innovatiemanagement'."
26,185,Innoveren in zorg en welzijn,"Innoveren, zorgtechnologie, implementeren, ontwerpen, gezondheid",0.829365,0.182627,Breda,30,NLQF6,"['innoveren', 'zorgtechnologie', 'implementeren', 'ontwerpen', 'gezondheid']","[technologie, zorg, zorgtechnologie]","Deze module sluit erg goed aan bij je interesses in 'technologie', 'zorg' en 'zorgtechnologie', zoals die terugkomen in 'Innoveren in zorg en welzijn'."
188,379,Creative AI,"AI, creatieve sector, makerschap, originaliteit, ethiek, performatieviteit",0.668177,0.147134,Breda en Den Bosch,30,NLQF6,"['ai', 'creatieve', 'sector', 'makerschap', 'originaliteit', 'ethiek', 'performatieviteit']",[creatieve],"Je interesse in 'creatieve' komt duidelijk terug in 'Creative AI', waardoor deze module goed bij je aansluit."


In [72]:
from typing import List, Optional
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_modules_with(
    df: pd.DataFrame,
    vectorizer,
    X,
    feature_names,
    student_profile: str,
    top_n: int = 5,
    studycredit: Optional[int] = None,
    level: Optional[str] = None,
    location: Optional[str] = None
) -> pd.DataFrame:
    """
    Zelfde idee als recommend_modules, maar gebruikt de meegegeven
    vectorizer, X en feature_names. Handig voor experimenten met TF-IDF.
    """
    filtered_df = df.copy()

    # Filter op studiepunten, niveau, locatie
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]

    if level is not None:
        filtered_df = filtered_df[filtered_df["level"].isin([level])]

    if location is not None:
        filtered_df = filtered_df[
            filtered_df["location"].str.contains(location, case=False, na=False)
        ]

    # Niks over? -> lege tabel terug
    if filtered_df.empty:
        return pd.DataFrame(
            columns=[
                "id", "name", "shortdescription", "similarity",
                "similarity_raw", "location", "studycredit",
                "level", "module_tags", "match_terms", "reason"
            ]
        )

    # Studentprofiel vectoriseren
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    # Bijbehorende rijen uit X pakken
    row_indices = filtered_df.index.to_numpy()
    X_filtered = X[row_indices]

    # Cosine similarity
    sims = cosine_similarity(student_vec, X_filtered).flatten()

    filtered_df = filtered_df.copy()
    filtered_df["similarity_raw"] = sims

    # Normaliseren naar 0–1 binnen deze selectie
    max_sim = sims.max()
    if max_sim > 0:
        norm_sims = sims / max_sim
    else:
        norm_sims = sims

    filtered_df["similarity"] = norm_sims

    # Sorteren en top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    match_terms_list: List[List[str]] = []
    reasons: List[str] = []

    for idx in top.index:
        # positie van deze rij binnen X_filtered
        pos = np.where(row_indices == idx)[0][0]
        module_vec = X_filtered[pos]

        terms = extract_match_terms(student_vec, module_vec, feature_names)
        match_terms_list.append(terms)

        reasons.append(
            build_reason(
                match_terms=terms,
                module_name=top.at[idx, "name"],
                score=top.at[idx, "similarity"]
            )
        )

    top = top[[
        "id",
        "name",
        "shortdescription",
        "similarity",
        "similarity_raw",
        "location",
        "studycredit",
        "level",
        "module_tags",
    ]].copy()

    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top.reset_index(drop=True)

In [73]:
tfidf_configs = {
    "baseline": TfidfVectorizer(),
    "ngrams_1_2": TfidfVectorizer(ngram_range=(1, 2)),
    "pruned_vocab": TfidfVectorizer(
        ngram_range=(1, 2),
        max_df=0.8,      # woorden die in >80% van de modules zitten eruit
        min_df=2,        # woorden die maar 1x voorkomen eruit
        max_features=5000
    ),
    "sublinear": TfidfVectorizer(
        ngram_range=(1, 2),
        sublinear_tf=True
    ),
}

In [ ]:
student_profile = """
Ik heb een achtergrond in de zorg en ik ben geïnteresseerd in hoe technologie kan helpen om
zorg en welzijn slimmer te organiseren. Smart health, e-health en innovatieve oplossingen
voor ouderen spreken me erg aan.
"""

for name, vec in tfidf_configs.items():
    print("=== Config:", name, "===\n")

    # Vectorizer fitten op de clean_text van alle modules
    X_exp = vec.fit_transform(df["clean_text"])
    feature_names = vec.get_feature_names_out()

    # Aanbevelingen genereren met deze vectorizer
    result = recommend_modules_with(
        df=df,
        vectorizer=vec,
        X=X_exp,
        feature_names=feature_names,
        student_profile=student_profile,
        top_n=5,
        # studycredit=15,       # pas aan of zet op None
        # level="NLQF5",        # pas aan of zet op None
        # location="Breda"      # pas aan of zet op None
    )

    display(result[["name", "similarity", "similarity_raw" "match_terms", "reason"]])
    print("\n\n")

=== Config: baseline ===



,name,similarity,match_terms,reason
0,Technologie die ècht werkt: innovatie in zorg en welzijn,1.000000,"[health, smart, technologie, welzijn, zorg]","Deze module sluit erg goed aan bij je interesses in 'health', 'smart', 'technologie', 'welzijn' en 'zorg', zoals die terugkomen in 'Technologie die ècht werkt: innovatie in zorg en welzijn'."
1,Zorg dichtbij,0.743758,"[technologie, zorg]","Omdat 'technologie' en 'zorg' centraal staan in 'Zorg dichtbij', past deze module goed bij jouw interesses."
2,Acuut Complexe Zorg,0.697799,"[welzijn, zorg]","Je interesse in 'welzijn' en 'zorg' komt duidelijk terug in 'Acuut Complexe Zorg', waardoor deze module goed bij je aansluit."
3,Healthcare Innovation Design,0.681765,"[welzijn, zorg]","Omdat 'welzijn' en 'zorg' centraal staan in 'Healthcare Innovation Design', past deze module goed bij jouw interesses."
4,Module 3.4 Duurzaam gebouwde omgeving - Gezondheid en zorg,0.671563,"[welzijn, zorg]","Omdat 'welzijn' en 'zorg' centraal staan in 'Module 3.4 Duurzaam gebouwde omgeving - Gezondheid en zorg', past deze module goed bij jouw interesses."





=== Config: ngrams_1_2 ===



,name,similarity,match_terms,reason
0,Technologie die ècht werkt: innovatie in zorg en welzijn,1.000000,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Deze module sluit erg goed aan bij je interesses in 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn', zoals die terugkomen in 'Technologie die ècht werkt: innovatie in zorg en welzijn'."
1,Healthcare Innovation Design,0.801078,"[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Healthcare Innovation Design', waardoor deze module erg goed bij je aansluit."
2,Acuut Complexe Zorg,0.734733,"[welzijn, zorg, zorg welzijn]","Deze module sluit goed aan bij je interesses in 'welzijn', 'zorg' en 'zorg welzijn', zoals die terugkomen in 'Acuut Complexe Zorg'."
3,Module 3.4 Duurzaam gebouwde omgeving - Gezondheid en zorg,0.634175,"[welzijn, zorg, zorg welzijn]","Omdat 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Module 3.4 Duurzaam gebouwde omgeving - Gezondheid en zorg', past deze module goed bij jouw interesses."
4,Zorg dichtbij,0.623889,"[technologie, zorg]","Deze module sluit goed aan bij je interesses in 'technologie' en 'zorg', zoals die terugkomen in 'Zorg dichtbij'."





=== Config: pruned_vocab ===



,name,similarity,match_terms,reason
0,Technologie die ècht werkt: innovatie in zorg en welzijn,1.000000,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Deze module sluit erg goed aan bij je interesses in 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn', zoals die terugkomen in 'Technologie die ècht werkt: innovatie in zorg en welzijn'."
1,Healthcare Innovation Design,0.778143,"[welzijn, zorg, zorg welzijn]","Omdat 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Healthcare Innovation Design', past deze module goed bij jouw interesses."
2,Omdenken: anders werken in de zorg,0.764602,"[welzijn, zorg, zorg welzijn]","Omdat 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Omdenken: anders werken in de zorg', past deze module goed bij jouw interesses."
3,Zorg dichtbij,0.662248,"[technologie, zorg]","Omdat 'technologie' en 'zorg' centraal staan in 'Zorg dichtbij', past deze module goed bij jouw interesses."
4,Acuut Complexe Zorg,0.645573,"[welzijn, zorg, zorg welzijn]","Omdat 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Acuut Complexe Zorg', past deze module goed bij jouw interesses."





=== Config: sublinear ===



,name,similarity,match_terms,reason
0,Technologie die ècht werkt: innovatie in zorg en welzijn,1.000000,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Omdat 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie die ècht werkt: innovatie in zorg en welzijn', past deze module erg goed bij jouw interesses."
1,Langer thuis in de wijk,0.664127,"[oplossingen, ouderen, welzijn, zorg, zorg welzijn]","Deze module sluit goed aan bij je interesses in 'oplossingen', 'ouderen', 'welzijn', 'zorg' en 'zorg welzijn', zoals die terugkomen in 'Langer thuis in de wijk'."
2,Smart Health,0.658765,"[health, smart, smart health, technologie, zorg]","Deze module sluit goed aan bij je interesses in 'health', 'smart', 'smart health', 'technologie' en 'zorg', zoals die terugkomen in 'Smart Health'."
3,Werken vanuit je doelgroep,0.644189,"[organiseren, welzijn, zorg, zorg welzijn]","Deze module sluit goed aan bij je interesses in 'organiseren', 'welzijn', 'zorg' en 'zorg welzijn', zoals die terugkomen in 'Werken vanuit je doelgroep'."
4,Omdenken: anders werken in de zorg,0.639155,"[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Omdenken: anders werken in de zorg', waardoor deze module goed bij je aansluit."
